## Version 0.9.9.1 2021 03 10

### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 The code is also hosted at https://github.com/n4rus/tbot and is licensed under creative commons.
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [ ]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import datetime
import time

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [ ]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient
from datetime import datetime
from datetime import date
from time import time

nova_client = NovaClient(AccessKey, SecretKey)

### Importing List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [ ]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

sublcq = nova_client.subs()
sublcd = sublcq['data']

blccolumns = ['Currency', 'Sub-Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)

p = 0
subids = {}
sublcs = {}

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
    
    blcdataframe.append(
        pd.Series([
            subcy,
            sublc,
        ], index = blccolumns),
        ignore_index = True)
    blcdataframe = blcdataframe.append(
    pd.Series([
        subcy,
        sublc,
    ], index=blccolumns),
        ignore_index=True)
 
    p = p + 1

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
#blcdataframe

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
#bdataframe

### Symbol position, Dolar price and timestamp
For the execution to be done independently for each symbol, the below cell is needed:

In [ ]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol', 'Ask','Bid', 'Last Price','AVG Price', 'Shares to Buy','High 24h', 'Min 24h', 'Avg 24h', 'Timestamp', 'Requisition Timestamp']
e_dataframe = pd.DataFrame(columns = my_columns)

yi = 0

api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdat = requests.get(api_url).json()
for i in api_url:
    fdata = fdat['data']
    for yi in fdata[:9]:
        price = float(fdata['lastPrice'])
        ask = float(fdata['ask'])
        bid = float(fdata['bid'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        high = float(fdata['high24h'])
        mini = float(fdata['low24h'])
        mid = float((high - mini)/2) 
        abl = float((ask + bid + price) / 3)
    
        etsp = (fdata['timestamp'])
        tspmi = float(etsp/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
    
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")    
    
        yi = yi + 1
        e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
        e_dataframe = e_dataframe.append(
            pd.Series([
                        symbol, 
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                        ],
                    index = my_columns
                    ),
                    ignore_index = True)

e_dataframe

In [ ]:
print(fdat)

### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [ ]:
listtrades = nova_client.list_orders(symbol, 'FINISHED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Amount', 'Number of Shares to Buy','Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Timestamp']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tspmi = float(ldc['timestamp']/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                        tspm,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        sharestobuy,
                        price,
                        bs,
                        vx,
                        momentum,
                        tspm
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Last Orders Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [ ]:
#orders dataframe
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Last Orders Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp', 'Requisition Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)


for symbol in symbols:
    #from 'finished' to 'filled'
    listorders = nova_client.list_orders(symbol, 'FILLED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        #rework into a precise formula with momentum
        #This must free the mommentum table cell
        vxdata = tdata[s]
        vxi = vxdata['price']
        momentumprice = float(vxi)
        momentumraw = float(price) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")

        s = s + 1
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

In [ ]:
y = 1
d = o_dataframe['B/S']
for i in d:
    print(i)

In [ ]:
bsboo = o_dataframe['B/S']
bi = 1
ssell = 'SELL'
sbuy = 'BUY'
for bi in bsboo:
    if bsboo == ssell:
        sell = price
        buy = 'N/a'
    if bsboo == sbuy:
        buy = price 
        sell = 'N/a'
    bi = bi + 1

In [ ]:
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}

#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
# here the second case is uded, as well from this version on, first ones used common filled orders
my_columns = ['Sell', 'Buy', 'Profit', 'Timestamp', 'Requisition Timestamp']
boo_dataframe = pd.DataFrame(columns = my_columns)

listorders = nova_client.list_orders(symbol, 'FILLED')

bsboo = o_dataframe['B/S']
bi = 1
ssell = 'SELL'
sbuy = 'BUY'


for y in tdata:
    a = 9 
    b = 2
    gama = 0
    z = 0
    sti = 0
    filled = 'FILLED'
    statushd = {}
    ldata = listorders['data']
    stdata = o_dataframe['Status']
    bsdata = o_dataframe['B/S']
    lopdta = o_dataframe['Last Orders Price']
    price = float(ldc['price']) 

    tspi = float(ldc['timestamp']/1000)
    tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
    status = ldc['status']
        
    localtimestamp = int(time())
    dl = localtimestamp
    tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")
    
    
    sell = float(lopdty) - float(lopdty *(0.03))    
    buy = float(lopdty) - float(lopdty *(0.03))
    profit = sell - buy
    bm = buy - momentumprice
    
    for bi in bsboo:
        if bsboo == ssell:
            sell = price
            buy = 'N/a'
        if bsboo == sbuy:
            buy = price 
            sell = 'N/a'
        bi = bi + 1
    profit = sell[y] - buy[y]
    
    
#define booleans
#boolean dataframe
    ldata = listorders['data']
    bsdaty = bsdata[z]
    stdaty = stdata[z] 
    lopdty = lopdta[z]
    ldc = ldata[z]
    tspi = float(ldc['timestamp']/1000)
    tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
    sell = float(lopdty) - float(lopdty *(0.03))    
    buy = float(lopdty) - float(lopdty *(0.03))
    profit = sell - buy
    bm = buy - momentumprice
    y = y + 1 
    
    boolean_dataframe.append(
                    pd.Series(
                    [
                        bsdaty,
                        stdaty,
                        lopdty,
                        tsp,
                        tserver
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
    boolean_dataframe = boolean_dataframe.append(
                pd.Series([
                        bsdaty,
                        stdaty,
                        lopdty,
                        tsp,
                        tserver
                      ]
                ,index = my_columns), 
                ignore_index = True)

boolean_dataframe
if momentumprice > price and amount > 0.02 :
    sellprofit = math.floor(price*2/100)/1.00
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

#first block to be solved
if momentumprice < price and amount < 0.02:
    print("Market is not ready to sell")

if momentumprice < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    print("The quantity bought was ETH: ", bm)
    
    

#loop time count and do momentum profit auto cicle
#publish

#define booleans
#boolean dataframe

In [ ]:
#profit = sell - buy
sellprofit = math.floor(price*2/100)/1.00
spe = price + sellprofit
print(spe)

In [ ]:
import time
from datetime import date
from time import time 
#ref https://stackoverflow.com/questions/39294293/get-timestamp-in-seconds-from-pythons-datetime
dt = int(time())
tserver = datetime.fromtimestamp(dt).strftime("%H:%M:%S")

dy = int(time())
yserver = datetime.fromtimestamp(dt).strftime("%H:%M:%S")

if dt < dy:
    #tserver == tserver.replace(second=today.second + 2)
    print('tserver: ',tserver)
print(dt)

In [ ]:
localtimestamp = int(time())
dl = localtimestamp
tserver = datetime.fromtimestamp(dl).strftime("%H:%M:%S")
requisition_timestamp = f'requisition_timestamp: {dl}, {tserver}'
print(requisition_timestamp)

In [ ]:
tspi = float(ldc['timestamp']/1000)
tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
print(tsp)

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [ ]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
o_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [ ]:
writer.save()